# Import library

In [1]:
from selenium import webdriver
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import requests
import time
import re
import pandas as pd
pd.set_option('max_colwidth', 1000)

# Functions

In [ ]:
def scrape_table(xpath):
    # WebDriverWait(driver, 5).until(
    #     EC.presence_of_element_located((By.XPATH, '//*[@id="ContentPlaceHolder_contentPlaceholder_TC88F994D005_Col00"]/div[2]')))
    WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((By.XPATH, xpath)))
    # div_freshmen = driver.find_element_by_xpath(
    #     '//*[@id="ContentPlaceHolder_contentPlaceholder_TC88F994D005_Col00"]/div[2]')
    arr_tr = driver.find_elements_by_xpath(xpath)
    # text_freshmen = driver.execute_script("return arguments[0].innerText;", div_freshmen)
    column_names = [d.text for d in arr_tr[1].find_elements_by_tag_name('td')]
    df = pd.DataFrame(columns=column_names)
    for tr in arr_tr[2:]:
        arr_td = tr.find_elements_by_tag_name('td')
        temp_scholarship = arr_td[0].text.strip()
        try:
            temp_qualification = arr_td[1].text.strip()
        except:
            temp_qualification = None
        try:
            temp_period = arr_td[2].text.strip()
        except:
            temp_period = None
        try:
            temp_to_apply = arr_td[3].text.strip()
        except:
            temp_to_apply = None
        df = df.append({
            column_names[0]: temp_scholarship,
            column_names[1]: temp_qualification,
            column_names[2]: temp_period,
            column_names[3]: temp_to_apply
        }, ignore_index=True)

    return df

def scrape_page(slug):
    driver.get('https://nus.edu.sg/oam/scholarships/' + slug)
    time.sleep(1)
#     xpath = '//div[@name="divAccordion"]/div'
    xpath = '//div[@data-placeholder-label="Drag Contents Here (col-md-9 col-sm-8)"]/div'
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, xpath)))
    div_content = driver.find_element_by_xpath(xpath)
    content = driver.execute_script(
        "return arguments[0].innerText;", div_content)
    df_page = pd.DataFrame(columns=['subtitle', 'content'])
    for c in content.replace('\xa0', '').split('\n\n\n'):
        arr_c = c.strip().split('\n', 1)
        df_page = df_page.append({
            'subtitle': arr_c[0].strip(),
            'content': arr_c[1].strip()
        }, ignore_index=True)
    return df_page

# NUS

## Undergraduate

In [58]:
driver = webdriver.Chrome(
    '../../chromedriver_linux64/chromedriver')
url = 'https://www.nus.edu.sg/oam/scholarships/apply-for-scholarships'
driver.get(url)
time.sleep(1)
xpath = '//*[@id="ContentPlaceHolder_contentPlaceholder_TC88F994D005_Col00"]/div[2]/div[2]/table/tbody/*'
df_freshmen = scrape_table(xpath)
xpath = '//*[@id="ContentPlaceHolder_contentPlaceholder_TC88F994D005_Col00"]/div[4]/div/table/tbody/*'
df_current = scrape_table(xpath)
driver.close()

In [59]:
# have difficulty if tables have merged cells
df_freshmen

,"Scholarship\nFor more information on the respective scholarships, click here.",Nationality / Qualification,Application Period,To Apply
0,NUS Scholarships\n\n- NUS Global Merit Scholarship\n- NUS Merit Scholarship\n- NUS Sports Scholarship\n- NUS Performing & Visual Arts Scholarship,"Singapore Citizens,\nGCE 'A' Level, Local Polytechnic Diploma or equivalent qualifications (e.g. IB Diploma, NUSHS Diploma)","Application for AY2021 commences from 1 February to 19 March 2021. Only one common application is required.\n\nApplicants who are interested in the Wee Cho Yaw Future Leaders Award must apply for both NUS Scholarships and NUS Financial Aid, in the year of matriculation. Application for NUS Financial Aid is from 1 February to 1 April 2021.","Click here for useful links (including sample form, scholarship uploading facility, and scholarship referee report)."
1,Stephen Riady Young Entrepeneur Scholarship,None,None,None
2,Wee Cho Yaw Future Leaders Award,None,None,None
3,Lee Kuan Yew Scholarship to Encourage Upgrading (LKY-STEP) Award,"Singapore Citizens and Permanent Residents,\nLocal Polytechnic Diploma",Application for AY2021 commences from 1 May to 20 May 2021.,SMU is the Secretariat for LKY-STEP Award in AY2021/2022. Click here to apply directly with the Secretariat during the application period.
4,University Engineering Scholarship (UES),"Singapore Citizens and Permanent Residents,\nLocal Polytechnic Diploma",Application for AY2021 commences from 3 May to 21 May 2021.,SIT is the Secretariat for UES in AY2021/2022. Click here to apply directly with the Secretariat during the application period.
5,ASEAN Undergraduate Scholarship,"Citizens of ASEAN countries (excluding Singapore),\nHigh school qualifications eligible for admission to NUS",Based on admission application timeline.,All eligible students applying for undergraduate admission to NUS will be automatically considered. Separate scholarship applications are not required.
6,Science & Technology Undergraduate Scholarship,"Citizens of Asian countries (excluding Singapore),\nHigh school qualifications eligible for admission to NUS",All eligible students applying for undergraduate admission to NUS will be automatically considered. Separate scholarship applications are not required.,None
7,Scholarships administered by NUS Faculties/Offices,Varies,Application period varies.,Please refer to the respective pages for more information.\n- Singapore Citizens\n- Singapore Permanent Residents\n- International Students
8,Scholarships administered by external organisations,Varies,Application period varies.,None


In [60]:
df_current

,"Scholarship\nFor more information on the respective scholarships, click here.",Nationality,Application Period,To Apply
0,Scholarships administered by NUS Office of Financial Aid,Varies,Application will commence from 1 February to 1 April annually.,Please refer to the respective webpage for more information.
1,Scholarships administered by NUS Faculties/Offices,Varies,Application period varies.,None
2,Scholarships administered by external organisations,Varies,Application period varies.,None


In [139]:
all_pages = ['freshmen-singapore-citizens', 'freshmen-sprs',
             'freshmen-international-students', 'current-nus-undergraduates']
driver = webdriver.Chrome(
    '../../chromedriver_linux64/chromedriver')
df_all_pages = pd.DataFrame()
for p in all_pages:
    df_temp = scrape_page(p)
    df_temp['slug'] = p
    df_all_pages = df_all_pages.append(df_temp, ignore_index=True)
driver.close()

In [140]:
df_all_pages

,subtitle,content,slug
0,NUS Scholarships,Application for NUS Scholarships for AY2021 commences from 1 February to 19 March 2021. Only one common scholarship application is required. Click hereto apply.\n\nNUS Global Merit Scholarship\n\nNUS Merit Scholarship\n\nNUS Sports Scholarship\n\nNUS Performing & Visual Arts Scholarship,freshmen-singapore-citizens
1,Donated Scholarships,"Application for Donated Scholarships for AY2021 commences from 1 February to 19 March 2021. Only one common scholarship application is required. Click hereto apply.\n\nPlease note that applicants who wish to be considered for the Wee Cho Yaw Future Leaders Award must apply for both NUS Scholarships and NUS Financial Aid, in the year of matriculation.\n\nStephen Riady Young Entrepreneur Scholarship\n\nWee Cho Yaw Future Leaders Award",freshmen-singapore-citizens
2,LKY-STEP Award,Applications for the LKY-STEP Award for AY2021 commences from 1 May to 21 May 2021.\n\nThe Lee Kuan Yew Scholarship to Encourage Upgrading (LKY-STEP) Awardis open to Polytechnic Diploma holders from Singapore who are matriculating into a full-time undergraduate programme atNUS / NTU / SMU / SUTD / SIT / SUSS in the year of application. Clickherefor more information.,freshmen-singapore-citizens
3,University Engineering Scholarship,Applications for the UES for AY2021 commences from 3 May to 21 May 2021.\n\nThe University Engineering Scholarship (UES) is open to Polytechnic Diploma holders from Singapore who are matriculating into a full-time Engineering or Technology-related undergraduate programme at NUS / NTU / SUTD / SIT in the year of application. Clickherefor more information.,freshmen-singapore-citizens
4,Faculty Administered Scholarships,Chinese Studies Entrance Scholarship (Department of Chinese Studies)\n\nFaculty of Engineering Scholarships\n\nNesadevi Sandrasegara Scholarship (Faculty of Law)\n\nSchool of Design & Environment Scholarships\n\nSingapore Academy of Law Undergraduate Scholarship (Faculty of Law),freshmen-singapore-citizens
5,Scholarships Offered by External Organisations,"BCA Undergraduate Scholarship\n\nBenjy Grinberg Scholarship\n\nCIMB ASEAN Scholarship\n\nEvolve Warrior Scholarship\n\nHowe Yoon Chong PSA Scholarship\n\nLim Kim San Memorial Scholarship\n\nMENDAKI - Dr Abdul Aziz Ali Scholarship\n\nMENDAKI - Singapore Chinese Chamber of Commerce Foundation (MENDAKI-SCCCF) Scholarship\n\nMendaki Scholarship (University)\n\nRaffles Medical Group Scholarship\n\nSingapore Digital (SG:D) Scholarship\n\nSingapore General Hospital Nursing Local Sponsorship\n\nSingapore-Industry Scholarships (SgIS)\n\nSmart Nation Scholarship(Offered by CSA, GovTech and IMDA)\n\nSPD Study Award\n\nThe HDB Scholarship\n\nThe National Kidney Foundation (NKF) Sponsorship Programme\n\nThe Ngee Ann Kongsi Tertiary Awards",freshmen-singapore-citizens
6,Scholarships Administered by NUS,Eligible students will be considered for the following scholarships through their application for admission to NUS. No separate scholarship application is required.\n\nASEAN Undergraduate Scholarship (AUS)\n\nScience & Technology (S&T) Undergraduate Scholarship,freshmen-sprs
7,LKY-STEP Award,Applications for the LKY-STEP Award for AY2021 commences from 1 May to 21 May 2021.\n\nThe Lee Kuan Yew Scholarship to Encourage Upgrading (LKY-STEP) Awardis open to Polytechnic Diploma holders from Singapore who are matriculating into a full-time undergraduate programme atNUS / NTU / SMU / SUTD / SIT / SUSS in the year of application. Clickherefor more information.,freshmen-sprs
8,University Engineering Scholarship,Applications for the UES for AY2021 commences from 3 May to 21 May 2021.\n\nThe University Engineering Scholarship (UES) is open to Polytechnic Diploma holders from Singapore who are matriculating into a full-time Engineering or Technology-related undergraduate programme at NUS / NTU / SUTD / SIT in the year of application.Clickherefor more information.,freshmen-sprs
9,Faculty Administered Scholarships,Chi

## Graduate

In [31]:
driver = webdriver.Chrome(
    '../../chromedriver_linux64/chromedriver')
driver.get('https://nusgs.nus.edu.sg/scholarships/')
time.sleep(1)
xpath = '//*[@id="eael-filter-gallery-wrapper-56208ca"]/div[2]/*'
WebDriverWait(driver, 5).until(
    EC.presence_of_all_elements_located((By.XPATH, xpath)))
arr_div = driver.find_elements_by_xpath(xpath)
# df_grad = pd.DataFrame(columns=['link', 'award', 'requirement', 'type'])
df_grad = pd.DataFrame(columns=['link', 'content'])
counter = 1
for d in arr_div:
    ActionChains(driver).move_to_element(d).perform()
    link = d.find_element_by_tag_name('a').get_attribute('href')
#     xpath = '//*[@id="eael-filter-gallery-wrapper-56208ca"]/div[2]/div[{}]/div/a/div[2]/div[2]/div/p[8]'.format(
#         counter)
#     requirement = driver.find_element_by_xpath(xpath).text
#     xpath = '//*[@id="eael-filter-gallery-wrapper-56208ca"]/div[2]/div[{}]/div/a/div[2]/div[2]/div/p[9]'.format(
#         counter)
#     scholarship_type = driver.find_element_by_xpath(xpath).text
#     xpath = '//*[@id="eael-filter-gallery-wrapper-56208ca"]/div[2]/div[{}]/div/a/div[2]/div[2]/div/p[10]'.format(
#         counter)
#     award = driver.find_element_by_xpath(xpath).text
#     df_grad = df_grad.append({
#         'link': link,
#         'requirement': nationality,
#         'type': scholarship_type,
#         'award': award
#     }, ignore_index=True)
#     xpath = '//*[@id="eael-filter-gallery-wrapper-56208ca"]/div[2]/div[{}]'
#     div_content = driver.find_element_by_xpath(xpath)
    div_content = d.find_element_by_class_name('fg-item-content')
    content = driver.execute_script(
        "return arguments[0].innerText;", div_content)
    df_grad = df_grad.append({
        'link': link,
        'content': content.strip()
    }, ignore_index=True)
    counter += 1
driver.close()

In [32]:
df_grad

,link,content
0,https://nusgs.nus.edu.sg/scholarships-list/?nusrs,For all nationalities from selected faculties/schools.\n\nFull Tuition Subsidy\n\nMonthly Award: $2700 with CPF (Singaporeans)
1,https://nusgs.nus.edu.sg/scholarships-list/?cws,For International Students from Commonwealth countries and selected faculties/schools.\n\nBy Nomination Only\n\nFull Tuition Subsidy\n\nMonthly Award : $2000
2,https://nusgs.nus.edu.sg/scholarships-list/?pgf,"For all nationalities from selected faculties/schools.\n\nBy Nomination Only\n\nFull Tuition Subsidy\n\nMonthly Award: $3,500 with CPF (Singaporeans)"
3,https://nusgs.nus.edu.sg/scholarships-list/?tfa,For all nationalities from selected faculties/schools.\n\nPartial Tuition Subsidy
4,https://nusgs.nus.edu.sg/scholarships-list/?nss,For International Students from selected faculties/schools.\n\nFull Tuition Subsidy\n\nMonthly Award: $2000
5,https://nusgs.nus.edu.sg/scholarships-list/?asgs,For all nationalities from ISEP-NGS.\n\nFull Tuition Subsidy
6,https://nusgs.nus.edu.sg/scholarships-list/?dss,For all nationalities from ISEP-NGS.\n\nFull Tuition Subsidy\n\nMonthly Award: $3500 with CPF (Singaporeans)
7,https://nusgs.nus.edu.sg/scholarships-list/?cqts,For all nationalities from CQT.\n\nFull Tuition Subsidy\n\nMonthly Award: $2900 with CPF (Singaporeans)
8,https://nusgs.nus.edu.sg/scholarships-list/?csi,For all nationalities from CSI.\n\nFull Tuition Subsidy\n\nMonthly Award: $3400 with CPF (Singaporeans)
9,https://nusgs.nus.edu.sg/scholarships-list/?isep,For all nationalities. Administered by ISEP-NGS.\n\nFull Tuition Subsidy\n\nMonthly Award: $3500 with CPF (Singaporeans)


In [39]:
# detail
driver = webdriver.Chrome(
    '../../chromedriver_linux64/chromedriver')
arr_full_content = []
for url in df_grad['link'].unique():
    print(url)
    driver.get(url)
    time.sleep(1)
    xpath = '//div[@class="elementor-tab-content elementor-clearfix elementor-active"]'
    WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((By.XPATH, xpath)))
    div_content = driver.find_element_by_xpath(xpath)
    content = driver.execute_script(
        "return arguments[0].innerText;", div_content)
    arr_full_content.append(content.strip())
driver.close()

https://nusgs.nus.edu.sg/scholarships-list/?nusrs
https://nusgs.nus.edu.sg/scholarships-list/?cws
https://nusgs.nus.edu.sg/scholarships-list/?pgf
https://nusgs.nus.edu.sg/scholarships-list/?tfa
https://nusgs.nus.edu.sg/scholarships-list/?nss
https://nusgs.nus.edu.sg/scholarships-list/?asgs
https://nusgs.nus.edu.sg/scholarships-list/?dss
https://nusgs.nus.edu.sg/scholarships-list/?cqts
https://nusgs.nus.edu.sg/scholarships-list/?csi
https://nusgs.nus.edu.sg/scholarships-list/?isep
https://nusgs.nus.edu.sg/scholarships-list/?nusirp
https://nusgs.nus.edu.sg/scholarships-list/?nusmed
https://nusgs.nus.edu.sg/scholarships-list/?lkysps
https://nusgs.nus.edu.sg/scholarships-list/?nussris
https://nusgs.nus.edu.sg/scholarships-list/?yale
https://nusgs.nus.edu.sg/scholarships-list/?mbi
https://nusgs.nus.edu.sg/scholarships-list/?aidf
https://nusgs.nus.edu.sg/scholarships-list/?aisg
https://nusgs.nus.edu.sg/scholarships-list/?dso


In [40]:
df_grad['full_content'] = arr_full_content

In [41]:
df_grad

,link,content,full_content
0,https://nusgs.nus.edu.sg/scholarships-list/?nusrs,For all nationalities from selected faculties/schools.\n\nFull Tuition Subsidy\n\nMonthly Award: $2700 with CPF (Singaporeans),"The NUS Research Scholarship (“Scholarship”) is awarded to outstanding graduate students for research leading to a higher degree at the University. The Scholarship consists of a monthly stipend plus a tuition fee subsidy.\n\nEligibility\n\nThe Scholarship is open to candidates who meet the following criteria:\n\nhave graduated with an undergraduate degree with at least Second Class Honours (Upper division) / Honours (Distinction) or equivalent;\nhave been offered admission to / or already admitted to a full-time graduate research programme at NUS; and\nmust be eligible for MOE Subsidy1\n1Please click here for Eligibility Guidelines for MOE Subsidy \nAward Details\nA monthly stipend as follows:\n \tFOR PHD CANDIDATES:\tFOR MASTER’S CANDIDATES:\nSingapore Citizen2\tS$2,700\tS$2,500\nSingapore Permanent Resident\tS$2,200\tS$1,500\nInternational Student\tS$2,000\tS$1,500\n\n2With effect from 1 August 2015, Central Provident Fund (CPF) contributions shall be provided at a rate pegged to..."
1,https://nusgs.nus.edu.sg/scholarships-list/?cws,For International Students from Commonwealth countries and selected faculties/schools.\n\nBy Nomination Only\n\nFull Tuition Subsidy\n\nMonthly Award : $2000,"The Commonwealth Scholarship (“Scholarship”) is awarded to outstanding graduate students from Commonwealth countries for research leading to a higher degree at the University. This Scholarship is for new, incoming students who are not Singapore citizens or permanent residents. The Scholarship consists of a monthly stipend plus a tuition fee subsidy. A number of students are selected each semester by the University for the award. \n\nEligibility\n\nThe Scholarship is open to candidates who meet the following criteria:\n\nnationals from any Commonwealth countries except for Singapore Citizens and Singapore Permanent Residents;\nhave graduated with an undergraduate degree with at least Second Class Honours (Upper Division)/Honours (Distinction) or equivalent;\nat the time of award of the Scholarship, must have been offered admission to a full-time graduate research programme at NUS; and\nmust be eligible for MOE Subsidy1\n1Please click here for Eligibility Guidelines for MOE Subsidy \..."
2,https://nusgs.nus.edu.sg/scholarships-list/?pgf,"For all nationalities from selected faculties/schools.\n\nBy Nomination Only\n\nFull Tuition Subsidy\n\nMonthly Award: $3,500 with CPF (Singaporeans)","The President’s Graduate Fellowship (”Fellowship”) is awarded to PhD candidates who show exceptional promise or accomplishment in research. A number of PhD candidates are selected each semester by the University for the award. A number of Fellowships are donor-funded: President’s Graduate Fellowship (funded by Lee Kong Chian Scholarship) and President’s Graduate Fellowship (funded by Jasmine Scholarship)\n\nEligibility\n\nThe Fellowship is open to PhD candidates who meet the following criteria:\n\nExemplary academic record. Top performance in undergraduate / postgraduate studies, graduation from top programs, and other academic/research achievements.\nStrong research potential. Potential to be a future leader in their respective fields based on the strength and originality of research proposal, credible signs of research ability, and expertise/interests that might lead to a path-breaking piece of work. Research interests that are aligned and/or complementary with the strengths of N..."
3,https://nusgs.nus.edu.sg/scholarships-list/?tfa,For all nationalities from selected faculties/schools.\n\nPartial Tuition Subsidy,"The NUS Tuition Fee Allowance (“Award”) is awarded to outstanding graduate students for research leading to a higher degree at the University. The Award does not consist of a monthly stipend.\n\nEligibility\n\nThe Award is open to candidates who

# NTU

## Undergraduate

In [26]:
# freshmen
driver = webdriver.Chrome(
    '../../chromedriver_linux64/chromedriver')
arr_url = ['https://www.ntu.edu.sg/admissions/undergraduate/scholarships/freshmen']
df_freshmen = pd.DataFrame()
for url in arr_url:
    driver.get(url)
    time.sleep(1)

    arr_xpath = ['//*[@id="Content_C010_Col00"]',
                 '//*[@id="Content_C010_Col01"]']
    arr_type = []
    for xpath in arr_xpath:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, xpath)))
        arr_type.append(driver.find_element_by_xpath(
            xpath).get_attribute('data-placeholder-label'))
        time.sleep(1)

    arr_xpath = ['//*[@id="Content_C011_Col00"]',
                 '//*[@id="Content_C013_Col00"]']
    arr_title = []
    for xpath in arr_xpath:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, xpath)))
        div_title = driver.find_element_by_xpath(xpath)
        arr_title.append(div_title.get_attribute('data-placeholder-label'))
        div_title.click()
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 700)")
    arr_title = arr_title + arr_title

    xpath = '//ul[@class="simple-list__list"]'
    WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((By.XPATH, xpath)))
    arr_ul = driver.find_elements_by_xpath(xpath)
    counter = 0
    for ul in arr_ul:
        arr_li = ul.find_elements_by_tag_name('li')
        for li in arr_li:
            link = li.find_element_by_tag_name('a').get_attribute('href')
            name = driver.execute_script(
                "return arguments[0].innerText;", li).strip()
            df_freshmen = df_freshmen.append({
                'url': url,
                'type': arr_type[counter // 2],
                'title': arr_title[counter],
                'link': link,
                'name': name
            }, ignore_index=True)
        counter += 1
driver.close()

In [27]:
df_freshmen

,link,name,title,type,url
0,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/renaissance-engineering-programme-scholarship,Renaissance Engineering Programme Scholarship (REPS),​Administered by NTU,Singapore Citizens and Singapore Permanent Residents,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/freshmen
1,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/university-scholars-programme,NTU-University Scholars Programme (NTU-​USP) Scholarship,​Administered by NTU,Singapore Citizens and Singapore Permanent Residents,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/freshmen
2,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/nanyang-scholarship-(cn-yang-scholars-programme),Nanyang Scholarship (CN Yang Scholars Programme),​Administered by NTU,Singapore Citizens and Singapore Permanent Residents,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/freshmen
3,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/nanyang-scholarship,Nanyang Scholarship,​Administered by NTU,Singapore Citizens and Singapore Permanent Residents,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/freshmen
4,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/college-scholarship,College Scholarship,​Administered by NTU,Singapore Citizens and Singapore Permanent Residents,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/freshmen
5,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/school-scholarship,School Scholarship,​Administered by NTU,Singapore Citizens and Singapore Permanent Residents,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/freshmen
6,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/ewbarkerscholarship,E. W. Barker Scholarship,​Administered by NTU,Singapore Citizens and Singapore Permanent Residents,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/freshmen
7,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/hass-scholarship,"Humanities, Arts, and Social Sciences Scholarship ​",​Administered by NTU,Singapore Citizens and Singapore Permanent Residents,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/freshmen
8,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/irene-tan-liang-kheng-scholarship,Irene Tan Liang Kheng Scholarship,​Administered by NTU,Singapore Citizens and Singapore Permanent Residents,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/freshmen
9,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/lkcmedicine-scholarship,LKCMedicine Scholarship,​Administered by NTU,Singapore Citizens and Singapore Permanent Residents,https://www.ntu.edu.sg/admissions/undergraduate/scholarships/freshmen


In [41]:
# current undergraduate
driver = webdriver.Chrome(
    '../../chromedriver_linux64/chromedriver')
url = 'https://www.ntu.edu.sg/admissions/undergraduate/scholarships/existing-student'
driver.get(url)
xpath = '//*[@id="Content_C010_Col00"]/div/*'
WebDriverWait(driver, 5).until(
    EC.presence_of_all_elements_located((By.XPATH, xpath)))
arr_button = driver.find_elements_by_xpath(xpath)
time.sleep(1)  # click close for cookies
for b in arr_button:
    b.click()
    driver.execute_script("window.scrollTo(0, 300)")
arr_xpath = ['//*[@id="Content_C011_Col00"]',
             '//*[@id="Content_C027_Col00"]', '//*[@id="Content_C013_Col00"]']
df_admin_ntu = pd.DataFrame()
for xpath in arr_xpath:
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, xpath)))
    div_content = driver.find_element_by_xpath(xpath)
    content = driver.execute_script(
        "return arguments[0].innerText;", div_content)
    if 'C011' in xpath:
        eligibility = content
    elif 'C027' in xpath:
        benefit = content
    elif 'C013' in xpath:
        procedure = content
df_admin_ntu = df_admin_ntu.append({
    'Eligibility': eligibility.split('\n', 1)[1].strip(),
    'Benefits of Award': benefit.split('\n', 1)[1].strip(),
    'Application Procedures': procedure.split('\n', 1)[1].strip(),
    'Type': 'Scholarships Administered by NTU'
}, ignore_index=True)

xpath = '//*[@id="Content_C025_Col00"]/div/div/ul/*'
WebDriverWait(driver, 5).until(
    EC.presence_of_all_elements_located((By.XPATH, xpath)))
arr_li = driver.find_elements_by_xpath(xpath)
df_admin_ext = pd.DataFrame()
for li in arr_li:
    link = li.find_element_by_tag_name('a').get_attribute('href')
    name = driver.execute_script(
        "return arguments[0].innerText;", li)
    df_admin_ext = df_admin_ext.append({
        'Link': link,
        'Name': name,
        'Type': 'Scholarships Administered by External Organisations'
    }, ignore_index=True)

# driver.close()

In [42]:
df_admin_ntu

,Application Procedures,Benefits of Award,Eligibility,Type
0,"Application for AY21-22 will open from 1 July to 31 July 2021.\nApply online via StudentLink\nUnder “Financial Assistance”, please select “Application for Scholarship (Merit Based)”.\nFreshmen are to apply two days after you have matriculated online.\nOnly 1 application is required regardless of the scholarship. Applicants are not required to indicate the scholarships they are interested in. Candidates invited for an interview will be informed of the scholarship(s) they have been shortlisted for. \nTo be considered for a need-based scholarship, after submitting your online application form, you will be provided with a Document Checklist. You are required to complete the Document Checklist and upload the required supporting documents (ID and income documents). \nThere is no need to submit any ID or income documents if you are not interested in the need-based scholarships.\nScholarship interviews will take place in August and September. All applicants will be updated on the outcome ...","The value of the scholarship ranges from $3,000 to $16,000.\nThe scholarship is tenable for one academic year.","Current undergraduates or freshmen on subsidised tuition fees who have performed well academically and meet the criteria as specified by the donor of the scholarship. \nScholarships that are need-based will require supporting documents such as ID and income documents to be submitted during the application process. \nDepending on the donor, scholarship recipients may be required to intern at the donor’s organisation.\nThe scholarship recipient may not hold any other scholarships/bursaries/awards concurrently unless otherwise informed.\nInformation on the donor and scholarship",Scholarships Administered by NTU


In [43]:
df_admin_ext

,Link,Name,Type
0,https://www.a-star.edu.sg/Scholarships/For-Undergraduate-Studies/Overview,A*STAR Scholarships,Scholarships Administered by External Organisations
1,http://www.ago.gov.sg/careers/scholarship,AGO Auditing Service Scholarship,Scholarships Administered by External Organisations
2,https://www.spd.org.sg/asia-pacific-breweries-foundation-scholarship-for-persons-with-disabilities/,Asia Pacific Breweries Foundation Scholarship For Persons With Disabilities,Scholarships Administered by External Organisations
3,http://www3.ntu.edu.sg/oad2/website_files/Scholarships/BTP_Scholarship.pdf,Bishan-Toa Payoh GRC Community Scholarship,Scholarships Administered by External Organisations
4,https://www.psc.gov.sg/Scholarships/public-sector-scholarships/browse-by-scholarship/central-provident-fund-board-cpfb-mid-term-local-undergraduate-scholarship-CPFB,CPF Board Scholarship Mid-Term Local Undergraduate Scholarship,Scholarships Administered by External Organisations
5,https://www.csit.gov.sg/internships-scholarships/csit-undergraduate-scholarship,Centre for Strategic Infocomm Technologies (CSIT) Undergraduate Scholarship Award,Scholarships Administered by External Organisations
6,https://www.hycpsascholarship.com/,Howe Yoon Chong PSA Scholarship,Scholarships Administered by External Organisations
7,http://www3.ntu.edu.sg/oad2/website_files/Scholarships/JAL_Sch.pdf,JAL Scholarship Program,Scholarships Administered by External Organisations
8,https://www.jcci.org.sg/scholarship/,JCCI Singapore Foundation Scholarships,Scholarships Administered by External Organisations
9,http://overseas.mofa.go.kr/sg-en/brd/m_2435/view.do?seq=761193&fbclid=IwAR2BrvvgxhPyr0QmPX9nn5TPOxTzUt82NklUACmHCad3pknLJDynmA7pXLY,Korean Government Scholarship Program for Graduate Degree,Scholarships Administered by External Organisations


## Graduate

In [52]:
driver = webdriver.Chrome(
    '../../chromedriver_linux64/chromedriver')
url = 'https://www.ntu.edu.sg/admissions/graduate/financialmatters/scholarships'
driver.get(url)
df_grad = pd.DataFrame()
for i in range(3):
    xpath = '//*[@id="Content_C011_Col0{}"]'.format(i)
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, xpath)))
    temp_div = driver.find_element_by_xpath(xpath)
    link = temp_div.find_element_by_tag_name('a').get_attribute('href')
    name = driver.execute_script(
        "return arguments[0].innerText;", temp_div)
    df_grad = df_grad.append({
        'link': link,
        'name': name,
        'type': 'Graduate Research Programme'
    }, ignore_index=True)

xpath = '//*[@id="Content_C042_Col00"]/div/ul/*'
WebDriverWait(driver, 5).until(
    EC.presence_of_all_elements_located((By.XPATH, xpath)))
arr_li = driver.find_elements_by_xpath(xpath)
for li in arr_li:
    link = li.find_element_by_tag_name('a').get_attribute('href')
    name = driver.execute_script(
        "return arguments[0].innerText;", li)
    df_grad = df_grad.append({
        'link': link,
        'name': name,
        'type': 'Other'
    }, ignore_index=True)
driver.close()

In [53]:
df_grad

,link,name,type
0,https://www.ntu.edu.sg/admissions/graduate/financialmatters/scholarships/npgs,Nanyang President's Graduate Scholarship,Graduate Research Programme
1,https://www.ntu.edu.sg/admissions/graduate/financialmatters/scholarships/rss,NTU Research Scholarship,Graduate Research Programme
2,https://www.a-star.edu.sg/Scholarships/for-graduate-studies/singapore-international-graduate-award-singa,Singapore International Graduate Award (SINGA),Graduate Research Programme
3,,APEC Scholarship,Other
4,https://www.a-star.edu.sg/Scholarships/for-graduate-studies/a-star-graduate-scholarship-singapore,A*Star Graduate Scholarship,Other
5,,Economic Development Board Industrial Postgraduate Programme (EDB-IPP),Other
6,https://www.psc.gov.sg/Scholarships/lee-kuan-yew-scholarship,Lee Kuan Yew Scholarship,Other
7,,MSc (TIP) Financial Grants,Other
8,,Muhammad Ariff Ahmad (MAS) Postgraduate Scholarship,Other
9,,Nanyang Fellows Scholarship,Other


# Archive

In [ ]:
# # freshmen singapore citizens
# df_freshmen_singapore = scrape_page('freshmen-singapore-citizens')

In [117]:
# df_freshmen_singapore

In [118]:
# # freshmen SPRs
# df_freshmen_sprs = scrape_page('freshmen-sprs')

In [119]:
# df_freshmen_sprs

In [120]:
# # freshmen international students
# df_freshmen_int_students = scrape_page('freshmen-international-students')

In [121]:
# df_freshmen_int_students

In [122]:
# # current NUS undergraduates
# df_freshmen_nus_under = scrape_page('current-nus-undergraduates')

In [123]:
# df_freshmen_nus_under